In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.250:7077") \
        .appName("MitraRokni_Section_A")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .config("spark.cores.min", 1)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/21 21:53:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Question A.1 
In this assignment I am using the English to Finnish text books.

In [2]:
# A.1 Loading English transcripts from HDFS counting the number of lines from English language from HDFS:
English_text= spark_session.sparkContext.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.fi-en.en") # English version

# Counting the number of lines in the English book and the Swedish book
Number_Of_Line_In_English = English_text.count()
print("Number of lines in English version of book is: ",Number_Of_Line_In_English)

Number of lines in English version of book is:  1924942


In [3]:
# A.2 Loading Finnish transcripts from HDFS counting the number of lines from Finnish language from HDFS:
Finnish_text= spark_session.sparkContext.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.fi-en.fi") # Finnish version

# Counting the number of lines in the English book and the Swedish book
Number_Of_Line_In_Finnish = Finnish_text.count()
print("Number of lines in Finnish version of book is: ",Number_Of_Line_In_Finnish)


Number of lines in Finnish version of book is:  1924942


In [4]:
print(English_text.take(5))

['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You have requested a debate on this subject in the course of the next few days, during this part-session.', "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union."]


In [5]:
print(Finnish_text.take(5))

['Istuntokauden uudelleenavaaminen', 'Julistan perjantaina joulukuun 17. päivänä keskeytetyn Euroopan parlamentin istunnon avatuksi ja esitän vielä kerran vilpittömän toiveeni siitä, että teillä olisi ollut oikein mukava joululoma.', 'Kuten olette varmaan saattaneet huomata, vuodenvaihteeseen 2000 povattuja suuria tietokoneongelmia ei ilmennytkään. Sen sijaan todella kauheat luonnonkatastrofit koettelivat kansalaisia joissakin unionimme maissa.', 'Te olette esittäneet toiveen, että tästä asiasta keskusteltaisiin lähipäivinä tämän istuntojakson aikana.', 'Sillä välin toivoisin, kuten useampi kollega on minulle esittänytkin, että viettäisimme minuutin hiljaisuuden kaikkien niiden uhrien muistoksi, jotka saivat surmansa useita Euroopan unionin maita koetelleissa myrskyissä.']


In [6]:
# A.1.3 Verify that the line counts are the same for the two languages
print("Are the line counts the same for the two languages?")
if Number_Of_Line_In_English == Number_Of_Line_In_Finnish:
    print("Answer is >> YES. The line counts are the same for the two languages")
else:
    print("Answer is >> NO!The line counts are not the same for the two languages. \nIn English book there is: ", Number_Of_Line_In_English ," Lines"," \nIn Swedish book there is ",  Number_Of_Line_In_Swedish , "lines.")

Are the line counts the same for the two languages?
Answer is >> YES. The line counts are the same for the two languages


In [7]:
# A.1.4 Count the number of partitions 
print(English_text.getNumPartitions())# number of partition for English
print(Finnish_text.getNumPartitions())# number of partition for Finnish

3
3


Question A.2 >> Pre-processing

In [8]:
# A.2.1 part1 defining the function 
def Preprocessing_The_Text(line):
    """
    Pre-process the text from both RDDs: by doing the following:
            Lowercase the text
            Tokenize the text by space " " 
    """
    return line.lower().split(" ")


In [9]:
# A.2.1  Continuing part 1 ............................ 

preprocessed_english = English_text.map(Preprocessing_The_Text) # Apply preprocessing to the English text RDD

preprocessed_finnish = Finnish_text.map(Preprocessing_The_Text) # Apply preprocessing to the Finnish text RDD


In [10]:
# A.2.3 Inspect 10 entries from each of your RDDs to verify your pre-processing
print(preprocessed_english.take(10))
print(preprocessed_finnish.take(10))

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

[['istuntokauden', 'uudelleenavaaminen'], ['julistan', 'perjantaina', 'joulukuun', '17.', 'päivänä', 'keskeytetyn', 'euroopan', 'parlamentin', 'istunnon', 'avatuksi', 'ja', 'esitän', 'vielä', 'kerran', 'vilpittömän', 'toiveeni', 'siitä,', 'että', 'teillä', 'olisi', 'ollut', 'oikein', 'mukava', 'joululoma.'], ['kuten', 'olette', 'varmaan', 'saattaneet', 'huomata,', 'vuodenvaihteeseen', '2000', 'povattuja', 'suuria', 'tietokoneongelmia', 'ei', 'ilmennytkään.', 'sen', 'sijaan', 'todella', 'kauheat', 'luonnonkatastrofit', 'koettelivat', 'kansalaisia', 'joissakin', 'unionimme', 'maissa.'], ['te', 'olette', 'esittäneet', 'toiveen,', 'että', 'tästä', 'asiasta', 'keskusteltaisiin', 'lähipäivinä', 'tämän', 'istuntojakson', 'aikana.'], ['sillä', 'välin', 'toivoisin,', 'kuten', 'useampi', 'kollega', 'on', 'minulle', 'esittänytkin,', 'että', 'viettäisimme', 'minuutin', 'hiljaisuuden', 'kaikkien', 'niiden', 'uhrien', 'muistoksi,', 'jotka', 'saivat', 'surmansa', 'useita', 'euroopan', 'unionin', 'mai

In [11]:
# Second Solution to A.2.1 > part1 defining the function 
'''
def Conversion_Function(rdd):
    lowercase_rdd = rdd.map(lambda x: x.lower().split(' '))
    return lowercase_rdd
'''


"\ndef Conversion_Function(rdd):\n    lowercase_rdd = rdd.map(lambda x: x.lower().split(' '))\n    return lowercase_rdd\n"

In [12]:
# Second Solution to A.2.1 > part1 mapping function on both languages texts
'''
English_Text_prep = Conversion_Function(English_text)
Finnish_text_Prep = Conversion_Function(Finnish_text)
'''


'\nEnglish_Text_prep = Conversion_Function(English_text)\nFinnish_text_Prep = Conversion_Function(Finnish_text)\n'

In [13]:
# Second Solution to A.2.2 > part2 Inspection 10 entities
'''
#  Inspect 10 entries from each of your RDDs to verify your pre-processing
print(English_Text_prep.take(10))
print(Finnish_text_Prep.take(10))
'''

'\n#  Inspect 10 entries from each of your RDDs to verify your pre-processing\nprint(English_Text_prep.take(10))\nprint(Finnish_text_Prep.take(10))\n'

In [14]:
# A.2.3

line_count2_En= preprocessed_english.count()
line_count2_Finn= preprocessed_finnish.count()

# Verify that the line counts still match after the pre-processing 
print("\nDo the line counts still match after the pre-processing?")
if line_count2_En == line_count2_Finn:
    print("Answer is >> YES. The line counts in both languages still match after the pre-processing")
else:
    print("Answer is >> NO! The line counts in both languages DO NOT match after the pre-processing. \nIn English book there is: ", line_count2_En ," Lines"," \nIn Swedish book there is ", line_count2_Finn, "lines.")

print("\nLine count in preprocessed English RDD:", line_count2_En)
print("Line count in preprocessed Finnish RDD:", line_count2_Finn)


Do the line counts still match after the pre-processing?
Answer is >> YES. The line counts in both languages still match after the pre-processing

Line count in preprocessed English RDD: 1924942
Line count in preprocessed Finnish RDD: 1924942


Question A.3

In [15]:
# A.3.1 Compute the 10 most frequently
# I am using  preprocessed_english and preprocessed_finnish as RDDs of lists of words 
# FlatMap to break down the lists into individual words, then map each to (word, 1)
mapper_english = preprocessed_english.flatMap(lambda x: x)\
                .map(lambda w: (w, 1))\
                .reduceByKey(add)

# Compute the top 10 most frequently occurring words in the English corpus
top_ten_english = mapper_english.takeOrdered(10, key=lambda x: -x[1])

# Print the top 10 most frequently occurring words in the English corpus
print("Top 10 words in English corpus:")
for word, count in top_ten_english:
    print(f"{word}: {count}")
###################################################################################################################
# Add a separator for clarity
print("\n" + "#" * 100 + "\n")
####################################################################################################################
# Repeat the process for the Finnish text
mapper_finnish = preprocessed_finnish.flatMap(lambda x: x)\
                .map(lambda w: (w, 1))\
                .reduceByKey(add)

# Compute the top 10 most frequently occurring words in the Finnish corpus
top_ten_finnish = mapper_finnish.takeOrdered(10, key=lambda x: -x[1])

# Print the top 10 most frequently occurring words in the Finnish corpus
print("Top 10 words in Finnish corpus:")
for word, count in top_ten_finnish:
    print(f"{word}: {count}")



Top 10 words in English corpus:
the: 3631787
of: 1724191
to: 1600135
and: 1339069
in: 1127219
that: 830095
a: 803908
is: 785410
for: 553730
we: 548086

####################################################################################################



Top 10 words in Finnish corpus:
ja: 1249156
on: 1035956
että: 619655
euroopan: 257568
ei: 246268
myös: 178765
ovat: 161869
se: 152857
arvoisa: 149439
ole: 134745


A.3.2 Verify that your results are reasonable. 
>
It is entirely reasonable because these words, known as stopwords, are the most common and frequent in every language, particularly in English. Therefore, for more advanced and specialized analysis, data analysts eliminate these words and symbols during the preprocessing steps.

Question A.4
This Question contains 9 steps: 

In [16]:
# Step 1&2 > Indexing lines and Swapping the Key and Values.
english_indexed = preprocessed_english.zipWithIndex().map(lambda x: (x[1], x[0]))
print(english_indexed.take(10))
finnish_indexed = preprocessed_finnish.zipWithIndex().map(lambda x: (x[1], x[0]))
print(finnish_indexed.take(10))


[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']), (2, ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.']), (3, ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.']), (4, ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,',

[(0, ['istuntokauden', 'uudelleenavaaminen']), (1, ['julistan', 'perjantaina', 'joulukuun', '17.', 'päivänä', 'keskeytetyn', 'euroopan', 'parlamentin', 'istunnon', 'avatuksi', 'ja', 'esitän', 'vielä', 'kerran', 'vilpittömän', 'toiveeni', 'siitä,', 'että', 'teillä', 'olisi', 'ollut', 'oikein', 'mukava', 'joululoma.']), (2, ['kuten', 'olette', 'varmaan', 'saattaneet', 'huomata,', 'vuodenvaihteeseen', '2000', 'povattuja', 'suuria', 'tietokoneongelmia', 'ei', 'ilmennytkään.', 'sen', 'sijaan', 'todella', 'kauheat', 'luonnonkatastrofit', 'koettelivat', 'kansalaisia', 'joissakin', 'unionimme', 'maissa.']), (3, ['te', 'olette', 'esittäneet', 'toiveen,', 'että', 'tästä', 'asiasta', 'keskusteltaisiin', 'lähipäivinä', 'tämän', 'istuntojakson', 'aikana.']), (4, ['sillä', 'välin', 'toivoisin,', 'kuten', 'useampi', 'kollega', 'on', 'minulle', 'esittänytkin,', 'että', 'viettäisimme', 'minuutin', 'hiljaisuuden', 'kaikkien', 'niiden', 'uhrien', 'muistoksi,', 'jotka', 'saivat', 'surmansa', 'useita', 'eu

In [17]:
# Step 3 > Join the English RDD and Finnish RDD Together
lines_joined = english_indexed.join(finnish_indexed)
print(lines_joined.take(1))
print("\n\ncount of line join is : ", lines_joined.count())

[(30, (['we', 'know,', 'and', 'we', 'have', 'stated', 'as', 'much', 'in', 'very', 'many', 'resolutions', 'indeed,', 'including', 'specifically', 'during', 'the', 'last', 'plenary', 'part-session', 'of', 'last', 'year,', 'that', 'this', 'is', 'not', 'solely', 'a', 'legal', 'case', 'and', 'that', 'it', 'is', 'wrong', 'for', 'alexander', 'nikitin', 'to', 'be', 'accused', 'of', 'criminal', 'activity', 'and', 'treason', 'because', 'of', 'our', 'involvement', 'as', 'the', 'beneficiaries', 'of', 'his', 'findings.'], ['me', 'tiedämme', 'ja', 'olemme', 'todenneet', 'sen', 'todellakin', 'erittäin', 'monissa', 'päätöslauselmissa', '-', 'nimenomaan', 'viime', 'vuoden', 'viimeisessä', 'täysistunnossa', '-,', 'että', 'tämä', 'ei', 'ole', 'pelkästään', 'oikeustapaus', 'ja', 'on', 'väärin', 'syyttää', 'aleksandr', 'nikitiniä', 'rikollisuudesta', 'ja', 'petoksesta,', 'koska', 'me', 'asianosaisina', 'hyödymme', 'hänen', 'tuloksistaan.']))]




count of line join is :  1924942


In [18]:
# Srep 4> Filter to exclude line pairs that have an empty/missing “corresponding” sentence.
Step_4= lines_joined.filter(lambda x: (x[1][0][0] != '' and x[1][1][0]!= ''))
print (Step_4.count())
print (Step_4.take(5))

1918494


[(918, (['once', 'the', 'committee', 'on', 'economic', 'and', 'monetary', 'affairs', 'has', 'adopted', 'the', 'draft', 'report', 'unanimously', 'with', 'two', 'abstentions,', 'i', 'would', 'ask', 'that', 'we', 'make', 'full', 'use', 'of', 'this', 'opportunity,', 'which', 'we', 'have', 'ourselves', 'created,', 'in', 'plenary', 'sitting.'], ['koska', 'talous-', 'ja', 'raha-asioiden', 'valiokunta', 'hyväksyi', 'mietintöluonnoksen', 'yksimielisesti', 'kahden', 'äänestäessä', 'tyhjää,', 'pyydän,', 'että', 'panemme', 'täysistunnossa', 'täydellisesti', 'täytäntöön', 'tämän', 'päätöksen,', 'jonka', 'olemme', 'itse', 'täällä', 'esittäneet.'])), (1458, (['we', 'have', 'received', 'many', 'excellent', 'observations', 'and', 'contributions', 'which', 'give', 'us', 'the', 'material', 'to', 'make', 'a', 'cost-benefit', 'assessment', 'for', 'business.'], ['olemme', 'saaneet', 'monia', 'erinomaisia', 'kommentteja', 'ja', 'mielipiteitä,', 'joista', 'saamme', 'aineistoa', 'yrityksille', 'laadittavaan', 

In [19]:
# Step 5> Filter to Leave Only Pairs of Sentences with a Small Number of Words

X = 5  # I also tried with x=10 , but i think the 5 is a better trashold
Step_5= Step_4.filter(lambda x: len(x[1][0]) <= X and len(x[1][1]) <= X)
print(f" Number of Pairs of Sentences with less than {X} Words are:{Step_5.count()}")
print(Step_5.take(10))

 Number of Pairs of Sentences with less than 5 Words are:72700


[(790806, (['\xa0\xa0', '.'], ['\xa0\xa0', '.'])), (36636, (['first', 'part'], ['ensimmäinen', 'osa'])), (36702, (['question', 'no', '33', 'by', '(h-0298/00):'], ['kysymys', 'nro', '33', '(h-0298/00):'])), (94476, (['.'], ['.'])), (101118, (['we', 'are', 'to', 'burn', 'it.'], ['poltamme', 'sen.'])), (107676, (['.'], ['.'])), (115464, (['we', 'voted', 'against', 'this', 'paragraph.'], ['äänestimme', 'tätä', 'kohtaa', 'vastaan.'])), (134778, (['(parliament', 'adopted', 'the', 'resolution)'], ['(parlamentti', 'hyväksyi', 'päätöslauselman.)'])), (159048, (['.'], ['.'])), (181482, (['question', 'no', '17', 'by', '(h-0626/01):'], ['kysymys', 'nro', '17', '(h-0626/01):']))]


In [20]:
# Step 6 > Only pairs of sentences with an equal number of words in each sentence.
# Assuming Filtering_Small_Sentences is your RDD after filtering for small sentences
Step_6 = Step_5.filter(lambda x: len(x[1][0]) == len(x[1][1]))

# Now, print the count and take 10 examples from sentences_with_equal_num_of_words
print(Step_6.count())
print(Step_6.take(50))


33820


[(266640, (['individual', 'producer', 'responsibility.'], ['tuottajien', 'erillinen', 'vastuu.'])), (365946, (['i', 'have', 'never', 'believed', 'that.'], ['sellaiseen', 'en', 'ole', 'koskaan', 'uskonut.'])), (429822, (['it', 'must', 'stop.'], ['sen', 'on', 'loputtava.'])), (825828, (['(applause)'], ['(suosionosoituksia)'])), (644442, (['\xa0\xa0', '.'], ['\xa0\xa0', '.'])), (5514, (['.'], ['.'])), (53904, (['we', 'also', 'need', 'other', 'things.'], ['me', 'tarvitsemme', 'myös', 'muita', 'asioita.'])), (121386, (['secondly:', 'the', 'distribution', 'of', 'work.'], ['toinen', 'kysymys', 'on', 'työn', 'jakautuminen.'])), (162138, (['report', 'brok', '(a5-0209/2001)'], ['brokin', 'mietintö', '(a5-0209/2001)'])), (178020, (['we', 'need', 'far', 'greater', 'clarity.'], ['kaipaamme', 'entistä', 'paljon', 'suurempaa', 'selkeyttä.'])), (188652, (['(interruption', 'by', 'mr', 'schultz)'], ['(jäsen', 'schultz', 'keskeytti', 'puhujan.)'])), (202392, (['such', 'conditions', 'affect', 'us', 'all.'

In [21]:
# step 7 > For each sentence pair,map so that you pair each(inorder)word in the two sentences.We no longer need the line numbers.(hint:usepython’sbuiltinzip()function)

pair_words =Step_6.map(lambda x: (x[1][0],x[1][1]))
mapper = pair_words.flatMap(lambda x: list(zip(x[0],x[1])))
print (mapper.take(10))


[('(applause)', '(suosionosoituksia)'), ('subject:', 'aihe:'), ('diabetes', 'diabetes'), ('madam', 'arvoisa'), ('president,', 'puhemies,'), ('my', 'pyydän'), ('apologies.', 'anteeksi.'), ('why?', 'miksi?'), ('i', 'minä'), ('endorse', 'kannatan')]


In [24]:
# Step 8 > Use reduce to count the number of occurrences of the word-translation-pairs.

# Count occurrences of each word pair
Reducer = mapper.map(lambda pair: (pair, 1)).reduceByKey(add)
print (Reducer.take(10))


[(('(applause)', '(suosionosoituksia)'), 2803), (('occur.', 'tapahtumaan.'), 1), (('.', '.'), 6973), (('spectrum', '/'), 1), (('is', 'merkitsee'), 2), (('real', 'todellista'), 5), (('progress.', 'edistystä.'), 15), (('attracts', 'synnyttää'), 1), (('words.', 'sanoja.'), 7), (('not', 'heittää'), 1)]


In [25]:
# Step 9 > Print some of the most frequently occurring pairs of words.
Most_Freq_Words = Reducer.takeOrdered(50, key=lambda x: -x[1])

# Print the results
print("Most frequently occurring word pairs:")
for pair, count in Most_Freq_Words:
    print(pair, count)


Most frequently occurring word pairs:
('.', '.') 6973
('\xa0\xa0', '\xa0\xa0') 5085
('is', 'on') 3045
('(applause)', '(suosionosoituksia)') 2803
('report', 'mietintö') 1061
('closed.', 'päättynyt.') 844
('debate', 'keskustelu') 813
('this', 'tämä') 700
('that', 'se') 558
('that', 'tämä') 537
('is', 'ei') 524
('it', 'se') 499
('applause', 'suosionosoituksia') 444
('1.', '1.') 428
('2.', '2.') 423
('-', '-') 406
('not', 'ole') 403
('3.', '3.') 393
('are', 'ovat') 375
('and', 'ja') 358
('minutes', 'pöytäkirja') 356
('thank', 'paljon') 353
('see', 'ks.') 353
('why?', 'miksi?') 340
('must', 'on') 337
('written', 'kirjalliset') 284
('we', 'meidän') 281
('applause)', 'suosionosoituksia)') 278
('what', 'mitä') 260
('4.', '4.') 243
('this', 'se') 237
('142)', 'artikla)') 226
('received:', 'asiakirjat:') 225
('documents', 'vastaanotetut') 225
('we', 'me') 221
('has', 'on') 218
('(rule', '(142') 212
('5.', '5.') 201
('important.', 'tärkeää.') 192
('you,', 'kiitoksia,') 190
('statements', 'kannano

The translations seem reasonable for the most part, especially for straightforward word pairs like ('report', 'mietintö') and ('debate', 'keskustelu'), which align well with dictionary translations. However, some pairs, like ('is', 'ei'), highlight potential inaccuracies likely due to the simplistic method of assuming direct word order correspondence. This suggests that a more nuanced approach, perhaps involving better preprocessing to remove stopwords, punctuation, and special characters like '\xa0\xa0', would significantly improve the analysis. By refining the preprocessing steps after steps 5 or 6, we could minimize the inclusion of less meaningful content, thereby focusing on more significant word pairs for translation. Such improvements would likely yield a more accurate reflection of frequent and important word translations, enhancing our understanding of the language pair involve

In [26]:
spark_session.stop()